# Scrape Visa Requirement -- US Citizen

# 1. Dependencies

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

#from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests

import pymongo

In [5]:
# get chromedriver location
!which chromedriver

/usr/local/bin/chromedriver


# 2. Scrap Data

## 2.1 Launch Driver

In [6]:
# launch chromedriver -- get an empty page
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

## 2.2 Scrape Wiki Page for Visa Requirements for US Citizens

In [7]:
url = "https://en.wikipedia.org/wiki/Visa_requirements_for_United_States_citizens"
browser.visit(url)

In [94]:
tables = pd.read_html(browser.html)
visa_df = tables[0]

In [95]:
# drop the unnecesary cols
col_to_drop = ['Allowed stay']
visa_df.drop(col_to_drop, axis = 1, inplace = True)

# rename cols
visa_df.rename(columns={'Country':'country',
                        "Visa requirement":'visa_requirement',
                        "Notes (excluding departure fees)":'notes'},
              inplace = True)
visa_df.head()

,country,visa_requirement,notes
0,Afghanistan,Visa required[2][3],Visitors born in Afghanistan do not require a ...
1,Albania,Visa not required[5][6],NaN
2,Algeria,Visa required[8][9],Persons may be denied entry if entering with a...
3,Andorra,Visa not required[10],NaN
4,Angola,eVisa[13][14][15],Visitors who have been granted an online pre-v...


In [96]:
# clean up the citation marks
for i in range(len(visa_df)):
    visa_df.iloc[i,1] = visa_df.iloc[i,1].split('[')[0]

In [97]:
visa_df['visa_requirement'].value_counts()

Visa not required                  107
Visa required                       30
Visa on arrival                     19
eVisa / Visa on arrival             18
eVisa                               11
Travel restricted                    1
Entry Permit on arrival              1
Visitor's Permit on arrival          1
Visitor's permit on arrival          1
eVisa / Tourist card on arrival      1
Tourist Card required                1
Electronic Travel Authority          1
Online Visa                          1
Name: visa_requirement, dtype: int64

In [98]:
auto_visa_key_words = ["on arrival", 'eVisa', 'Electronic', 'Online']
restrict_list = ["Tourist Card required", "Travel restricted"]

for i in range(len(visa_df)):
    for word in auto_visa_key_words:
        if word in visa_df['visa_requirement'][i]:
            visa_df['visa_requirement'][i] = "eVisa/Visa on arrival"
            continue
    if visa_df['visa_requirement'][i] in restrict_list:
        visa_df['visa_requirement'][i] = "Travel restricted"

In [99]:
visa_df['visa_requirement'].value_counts()

Visa not required        107
eVisa/Visa on arrival     54
Visa required             30
Travel restricted          2
Name: visa_requirement, dtype: int64

In [87]:
visa_df.head()

,country,visa_requirement,notes
0,Afghanistan,Visa required,Visitors born in Afghanistan do not require a ...
1,Albania,Visa not required,NaN
2,Algeria,Visa required,Persons may be denied entry if entering with a...
3,Andorra,Visa not required,NaN
4,Angola,eVisa/Visa on arrival,Visitors who have been granted an online pre-v...


## 2.3 Read the GDP_by_Country.csv file

In [40]:
# The CSV file is in the resources directory
# The source website is https://data.worldbank.org/indicator/ny.gdp.mktp.cd
gdp_filename = "resources\GDP_by_country.csv"
gdp_df = pd.read_csv(gdp_filename)
gdp_df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.498883e+09,2.390503e+09,2.549721e+09,2.534637e+09,2.581564e+09,2.649721e+09,2.691620e+09,2.646927e+09,2.700559e+09,NaN
1,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,537777811.1,548888895.6,546666677.8,751111191.1,800000044.4,1.006667e+09,...,1.243909e+10,1.585657e+10,1.780428e+10,2.000162e+10,2.056105e+10,2.048487e+10,1.990711e+10,1.936264e+10,2.019176e+10,1.936297e+10
2,Angola,AGO,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,7.030716e+10,8.379950e+10,1.117900e+11,1.280530e+11,1.367100e+11,1.457120e+11,1.161940e+11,1.011240e+11,1.221240e+11,1.057510e+11
3,Albania,ALB,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.204421e+10,1.192695e+10,1.289087e+10,1.231978e+10,1.277628e+10,1.322825e+10,1.138693e+10,1.186135e+10,1.302506e+10,1.505888e+10
4,Andorra,AND,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3.660531e+09,3.355695e+09,3.442063e+09,3.164615e+09,3.281585e+09,3.350736e+09,2.811489e+09,2.877312e+09,3.013387e+09,3.236544e+09


In [41]:
gdp_df.drop(columns=["Country Code","Indicator Name","Indicator Code"], inplace=True)
gdp_df.head()

,Country Name,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.498883e+09,2.390503e+09,2.549721e+09,2.534637e+09,2.581564e+09,2.649721e+09,2.691620e+09,2.646927e+09,2.700559e+09,NaN
1,Afghanistan,537777811.1,548888895.6,546666677.8,751111191.1,800000044.4,1.006667e+09,1.400000e+09,1.673333e+09,1.373333e+09,...,1.243909e+10,1.585657e+10,1.780428e+10,2.000162e+10,2.056105e+10,2.048487e+10,1.990711e+10,1.936264e+10,2.019176e+10,1.936297e+10
2,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.030716e+10,8.379950e+10,1.117900e+11,1.280530e+11,1.367100e+11,1.457120e+11,1.161940e+11,1.011240e+11,1.221240e+11,1.057510e+11
3,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.204421e+10,1.192695e+10,1.289087e+10,1.231978e+10,1.277628e+10,1.322825e+10,1.138693e+10,1.186135e+10,1.302506e+10,1.505888e+10
4,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.660531e+09,3.355695e+09,3.442063e+09,3.164615e+09,3.281585e+09,3.350736e+09,2.811489e+09,2.877312e+09,3.013387e+09,3.236544e+09


In [42]:
years = np.arange(1960,2018)
years_str = [str(yy) for yy in years]
gdp_df.drop(columns=years_str, inplace=True)
gdp_df.dropna(inplace=True)
gdp_df.rename(columns={"Country Name": "country", "2018":"GDP 2018"}, inplace=True)
gdp_df.set_index("country", inplace=True)
gdp_df.head()

,GDP 2018
country,
Afghanistan,1.936297e+10
Angola,1.057510e+11
Albania,1.505888e+10
Andorra,3.236544e+09
Arab World,2.781330e+12


## 2.4 Scrape Brilliant Maps page for Top 100 Tourist Destinations

In [14]:
# URL of page to be scraped
url = "https://brilliantmaps.com/top-100-tourist-destinations/"

# Retrieve page
response = requests.get(url)

# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(response.text, 'html.parser')

# Examine the results, then determine element that contains sought info
print(soup.prettify())

<!DOCTYPE html>
<html lang="en-US" prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# article: http://ogp.me/ns/article#">
 <head>
  <meta charset="utf-8"/>
  <meta content="Which city is the most popular international tourist destination? Surprisingly, it's not Paris, London or New York. Click to find out which city came out on top." name="description"/>
  <meta content="noodp,noydir" name="robots"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   Top 100 International Tourist Destination Cities By Country – Brilliant Maps
  </title>
  <link href="//fonts.googleapis.com" rel="dns-prefetch"/>
  <link href="https://brilliantmaps.com/feed/" rel="alternate" title="Brilliant Maps » Feed" type="application/rss+xml"/>
  <link href="https://brilliantmaps.com/comments/feed/" rel="alternate" title="Brilliant Maps » Comments Feed" type="application/rss+xml"/>
  <link href="https://brilliantmaps.com/top-100-tourist-destinations/feed/" rel="alternate" title

In [15]:
# Get the Top Destinations table, show the top 20 destinations
destinations = pd.read_html(response.text)
top_destinations=destinations[0]
top_destinations.head(20)

,Rank,City,Country,Tourists (Millions)
0,1,Hong Kong,Hong Kong,23.7
1,2,Singapore,Singapore,21.3
2,3,Bangkok,Thailand,15.8
3,4,London,UK,15.5
4,5,Macau,Macau,13.4
5,6,Kuala Lumpur,Malaysia,13.3
6,7,Shenzhen,China,12.1
7,8,New York City,USA,11.6
8,9,Antalya,Turkey,10.3
9,10,Paris,France,9.8


In [35]:
# Note that some countries have more than one city in the Global Top 100
# Find the most popular city to visit in each country
top_cities_by_country = top_destinations.loc[top_destinations.groupby(["Country"])["Rank"].idxmin()]
top_cities_by_country.rename(columns={"Country":"country", "City":"city","Tourists (Millions)":"tourists (mm)"}, inplace=True)
top_cities_by_country.set_index("country", inplace=True)
top_cities_by_country.drop(columns="Rank", inplace=True)
top_cities_by_country.head(20)

,city,tourists (mm)
country,,
Argentina,Buenos Aires,3.2
Australia,Sydney,2.7
Austria,Vienna,4.3
Azerbaijan,Baku,1.7
Bahrain,Manama,2.0
Belgium,Brussels,2.3
Brazil,Rio de Janeiro,1.8
Bulgaria,Sofia,4.1
Cambodia,Siem Reap,1.9


## 2.5 Read a json file with favorite dishes by country

In [2]:
# This json file comes from someone's github directory
# https://github.com/samayo/country-json/blob/master/src/country-by-national-dish.json

filepath = "resources\country-by-national-dish.json"
yummy_df = pd.read_json(filepath)
yummy_df.set_index("country", inplace=True)
yummy_df.head()

,dish
country,
Afghanistan,Kabuli Palaw
Albania,Tav‰ kosi
Algeria,Couscous
American Samoa,None
Andorra,None
